In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

from tqdm.notebook import trange, tqdm

In [2]:
env = gym.make('StockTradingEnvironment-v0', use_technical_indicators= [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
])

In [3]:
from src.agents.actor_critic.a2c import A2CAgent

In [4]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
                   use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    
    env.success_threshold =0.06 # 5%
    return env


In [5]:
agent=A2CAgent(environment, epochs=1, actor_learning_rate=0.000025,critic_learning_rate=0.000025,policy="CNN")
agent.load()
environment().success_threshold


* Models successfully loaded *



0.06

In [6]:
agent.hash

'ed75666dd5e47547b3de000d534a9270'

In [ ]:
agent.env.mode = "train"
agent.learning_log.episodes = 0
agent.learn(
    timesteps=-1, 
    log_every=100,
    success_threshold_lookback=1000,
    success_strict=True,
)

Episode * 100 * Worker * False * Moving Avg Reward is ==> 0.06214 * Last Reward was ==> 0.08256
Episode * 200 * Worker * False * Moving Avg Reward is ==> 0.05940 * Last Reward was ==> 0.08547
Episode * 300 * Worker * False * Moving Avg Reward is ==> 0.06442 * Last Reward was ==> 0.02403


In [ ]:
success = 0
n_tests = 0 #10000

scores = []
targets = []

for i in trange(n_tests):
    state = agent.env.reset(visualize=False,mode="test")
    step = 0
    score = 0
    reward = 0
    done = False
    targets.append((agent.env.episode_target-agent.env.initial_investment)/agent.env.initial_investment)

    while not done:
        agent.env.render()
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if agent.env.portfolio_value > agent.env.initial_investment:
                success +=1

        step+=1
        state = next_state
        score += reward
    
    #print(score,initial_portfolio, agent.env.portfolio_value)
    scores.append(score)
    
test_results_dataframe = pd.DataFrame([[
    n_tests,
    str(np.mean(scores)*100)+'%',
    str(np.mean(targets)*100)+'%',
    str(min(scores)*100)+'%',
    str(max(scores)*100)+'%',
    str((success/n_tests)*100) +'%'
]],
    columns=[
        '# Blind tests',
        '% Average portfolio return', 
        '% Desired portfolio return', 
        '% Historical minimum return',
        '% Historical maximum return', 
        '% Episode concluded with positive outcome'
    ]
)

# print(np.mean(scores))
# print(min(scores))
# print(max(scores))
# print(success)

test_results_dataframe.T

In [ ]:
#agent.results_writer.store_test_results(agent,test_results_dataframe)

In [ ]:
# from datetime import datetime

# test_results_dataframe.to_csv(agent.writer_log_directory+'/results__'+datetime.now().strftime("%Y%m%d-%H%M%S")+'.csv')

In [ ]:
success = 0
n_tests = 2

scores = []
targets = []


for i in trange(n_tests):
    state = agent.env.reset(visualize=True,mode="test")
    step = 0
    score = 0
    reward = 0
    done = False
    targets.append((agent.env.episode_target-agent.env.initial_investment)/agent.env.initial_investment)
    initial_portfolio = agent.env.portfolio_value

    while not done:
        agent.env.render()
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if agent.env.portfolio_value > agent.env.initial_investment:
                success +=1

        step+=1
        state = next_state
        score += reward
    
    #print(score,initial_portfolio, agent.env.portfolio_value)
    scores.append(score)

agent.env.close()